# Configurações iniciais

In [1]:
# Import de bibliotecas
from zipfile import ZipFile  # Leitura de arquivos .zip
import requests  # Requisição web
from io import BytesIO  # Leitura de arquivo em bytes
import numpy as np
import pandas as pd  # Manipulação de dados



# Configurações das bibliotecas
pd.options.display.max_columns = None

# Extração dos dados

In [2]:
# Extrair o arquivo excel da pasta ZIP diretamente do site de download
url = 'https://www.b3.com.br/data/files/57/E6/AA/A1/68C7781064456178AC094EA8/ClassifSetorial.zip'
response = requests.get(url=url)
with ZipFile(BytesIO(response.content)) as fold:
    file = fold.namelist()[0]
    with fold.open(file) as file:
        df = pd.read_excel(io=file)

# Transformação dos dados

In [3]:
df.columns = df.iloc[5].values
df = df.iloc[5:]
df = df.rename(columns={'SEGMENTO': 'NOME', 'LISTAGEM': 'CÓDIGO', np.nan: 'LISTAGEM'})[1:]
df.loc[(df['CÓDIGO'].isnull()), 'SEGMENTO'] = df.loc[(df['CÓDIGO'].isnull()), 'NOME']
df = df.dropna(how='all')
df = df[['CÓDIGO', 'NOME', 'SETOR ECONÔMICO', 'SUBSETOR', 'SEGMENTO', 'LISTAGEM']]
df['SETOR ECONÔMICO'].ffill(inplace=True)
df['SETOR ECONÔMICO'].astype('category')
df['SUBSETOR'].ffill(inplace=True)
df['SUBSETOR'].astype('category')
df['SEGMENTO'].ffill(inplace=True)
df['SEGMENTO'].astype('category')
df = df.loc[(df.CÓDIGO!='CÓDIGO') & (df.CÓDIGO!='LISTAGEM') & (~df.CÓDIGO.isnull())]
df.to_parquet(path='./tickers_por_setor.parquet')

In [4]:
display(df)

,CÓDIGO,NOME,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM
8,RRRP,3R PETROLEUM,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
9,CSAN,COSAN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
10,ENAT,ENAUTA PART,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
11,RPMG,PET MANGUINH,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN
12,PETR,PETROBRAS,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2
...,...,...,...,...,...,...
555,PPAR,POLPAR,Outros,Outros,Outros,NaN
556,PRPT,PROMPT PART,Outros,Outros,Outros,NaN
557,OPSE,SUDESTE S/A,Outros,Outros,Outros,NaN
558,OPTS,SUL 116 PART,Outros,Outros,Outros,NaN
